In [1]:
pip install pycryptodome cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.primitives import asymmetric, serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from Crypto.PublicKey import RSA
from Crypto.Cipher import DES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Util.number import getPrime
from Crypto.Util.Padding import pad, unpad
import socket
import base64
import time
import hashlib

class RSACrypto:
    def __init__(self, key_size=2048, public_exponent=65537, prime_size=1024, hash_function='SHA-256'):
        """Initializes all the evaluation parameters for the algorithm."""
        self.key_size = key_size
        self.public_exponent = public_exponent
        self.hash_function = hash_function
        self.prime_size = prime_size
        self.RSA_generation_time = None
        self.DES_generation_time = None
        self.generate_keys()
        
    def generate_keys(self):
        """Generates the public and private keys for RSA with the evaluation parameters as defined above."""
        start_time = time.time()
        key = RSA.generate(self.key_size, e=self.public_exponent)
        self.private_key = key.export_key()
        self.public_key = key.publickey().export_key()
        self.RSA_generation_time = time.time() - start_time

    def generate_DES_key(self):
        """Generates a single key for DES (Data Encryption Standard)."""
        start_time = time.time()
        key = get_random_bytes(8)
        end_time = time.time()
        self.DES_generation_time = end_time - start_time
        return key, self.DES_generation_time
    
    def public_encrypt(self, message):
        """Encrypts a secure message using the public key."""
        if self.public_key is None:
            raise ValueError("Public key not generated or loaded.")
        key = RSA.import_key(self.public_key)
        cipher = PKCS1_OAEP.new(key)
        encrypted_message = cipher.encrypt(message.encode())
        return encrypted_message
    
    def private_decrypt(self, encoded_message):
        """Decrypts the message that was encrypted above by using the private key."""
        if self.private_key is None:
            raise ValueError("Private key not available.")
        
        key = RSA.import_key(self.private_key)
        cipher = PKCS1_OAEP.new(key)
        decrypted_message = cipher.decrypt(encoded_message)
        return decrypted_message.decode()

    def get_security_parameters(self):
        """Returns the values of the evaluation parameters."""
        return {
            'Key Size': self.key_size,
            'Public Exponent': self.public_exponent,
            'Prime Size': self.prime_size,
            'Hash Function': self.hash_function,
            'RSA Key Generation Time': self.RSA_generation_time,
            'DES Key Generation Time': self.DES_generation_time
        }

In [3]:
print(hashlib.algorithms_available)

{'sm3', 'shake_256', 'md5-sha1', 'sha3_224', 'sha3_384', 'md5', 'blake2b', 'whirlpool', 'ripemd160', 'md4', 'blake2s', 'sha1', 'sha512', 'shake_128', 'sha384', 'sha512_256', 'sha224', 'sha256', 'sha512_224', 'sha3_256', 'mdc2', 'sha3_512'}


In [4]:
if __name__ == "__main__":
    rsa = RSACrypto(key_size=2048, hash_function='sha256')
    user_input = input("Please enter a message:")
    encrypted_message = rsa.public_encrypt(user_input)
    decrypted_message = rsa.private_decrypt(encrypted_message)
    print(encrypted_message)
    print(decrypted_message)
    print(rsa.get_security_parameters())
    key = get_random_bytes(8)
    DES_instance = DES.new(key, DES.MODE_ECB)
    encrypted_DES = DES_instance.encrypt(pad(user_input.encode('utf-8'), DES.block_size))
    decrypted_DES = unpad(DES_instance.decrypt(encrypted_DES), DES.block_size)
    print(encrypted_DES)
    print(decrypted_DES.decode('utf-8'))
    print(rsa.generate_DES_key())
    difference = rsa.RSA_generation_time - rsa.DES_generation_time
    print("Time Difference =",difference)
    print(f"RSA is a more secure algorithm than DES since the difference in generation time {difference}>0 implying that DES is faster and therefore less secure and DES will only accept two key parameters out of RSA (key size and generation time) since RSA generates prime numbers and incurs hash functions but DES does not do either of these.")

Please enter a message: would it be possible?


b'X\xb1\r\x05\x9c\x987\xe6\x88.\xea\x10\xf1\xf7\x01\xb7\xb4\xb6\x86\x8by\xc4\xedv\xe7\x9c\xe3A"\xb9\xd9\x02\xd2w\xec\x19G\x02I\'I9\x8d\xde\xcc\x1a\x13>6\x03\x14:\xbd\xdc(\xcb\x13\x80/w\xcf\x86t\x05\xa7\x7f4\x94\xee\x8b\xaa\xe6\xf7\xd1\xf3D\xb1qg"\x8e\xe7\xd4\xe9\xe5\x81\xb8\x86\xf7\xf5\xd0d\x12\xc5w7\xaf\xad\x86\xd2\xbdsW)\xb5\xde2\xdb\x9e\xc8^\xf9\x1dJ\x9e\x96:\xa9\xb9+Hp_J\xec5\xc8T]I\x1e\xbf\xe4\x05\xd7L\x03\x9a\x01\x8a*V\xa0\xd1\xa3j\xad\x83}\x9a\xeb\x99L\x115\xeeD\x9fv\xee\xb4}\x99)\x01L\xc5\x02\xd0\xb1\xa0\x11\xf9&m\xaa(#[W\xf0y\x81;\x94\xe6\x0c\x8a\x8a*7\x10\xed"\x7f>\xaa\xf5:\xcf\x0f\xc8\xc4\x0bt4J\x92\x85\xd8\xc8\xe5!y"\xba\x05\xf2\\\xae\xe5\x19\xa4\xd0T\xd1\xb0\xfd\x06N\x8b\x80\x1c2OCDo \xc9\x08Ue\x94TG\x19\x8a\xf64\x04\xf3\x023i"'
would it be possible?
{'Key Size': 2048, 'Public Exponent': 65537, 'Prime Size': 1024, 'Hash Function': 'sha256', 'RSA Key Generation Time': 3.4928901195526123, 'DES Key Generation Time': None}
b'\xa0C\x83\xc5\xc9|\xf9\xb5V:\xf6u\xb8de/U\xa4\n\x8e\